# **Tabular Boilerplate Notebook**

Tabular modeling takes data in the form of a table (like a spreadsheet or CSV), where the objective is to predict the value in one column based on the values in the other columns. This notebook will serve as a boilerplate handler for tabular data modeling, with sections laid out for each major step of the modeling process.

**REMEMBER**: This boilerplate is just that: boilerplate! It's a good idea to perform your own exploration in a manner that's specific to your given dataset. The default boilerplate uses the Titanic dataset from Kaggle as an example.

## **Setup**

This is the section for setting up your work environment, with boilerplate setups for a number of mainstream options. Don't see one you like? Add your own!

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

### **Imports and Installs**

Declare all your project's required imports and installs here:

In [2]:
# Installs
!pip install -Uqq fastai waterfallcharts treeinterpreter dtreeviz

     |████████████████████████████████| 204kB 7.7MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 


In [3]:
# Imports
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG
import numpy as np
import math
from scipy import stats

In [ ]:
# Statsmodels install (only necessary if module is needed and Colab gives trouble)
! pip install --upgrade Cython
! pip install --upgrade git+https://github.com/statsmodels/statsmodels

In [ ]:
# Set up some constraints here, if desired
pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

### **Colab Setup**

You can set up the Google Colab environment for data by mounting your Google Drive:

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/My Drive/

Mounted at /content/gdrive
/content/gdrive/My Drive




---



## **Data Collection**

Data specific to your current task can be collected here. There'll be different setups depending on where you're running this notebook, but the output here will be used for further data exploration.

### **Kaggle Datasets**

Get yourself started with some Kaggle datasets. First, set up your API credentials:

In [5]:
creds = {"username":"USERNAME_HERE","key":"API_KEY_HERE"}

Then set the credentials up in the `kaggle.json` so that Kaggle knows where to look for them in API calls:

In [6]:
!mkdir .kaggle
!mv .kaggle /root/
!touch /root/.kaggle/kaggle.json

!ls /root/.kaggle

kaggle.json


Then write the credentials to `kaggle.json` with the correct permissions setup to enable access to Kaggle datasets:

In [7]:
import json
import zipfile
import os

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(creds, file)

!chmod 600 /root/.kaggle/kaggle.json

In [8]:
!export KAGGLE_CONFIG_DIR=/root/.kaggle/kaggle.json

Before we finally install Kaggle itself:

In [ ]:
!pip install kaggle

from kaggle import api

Now let's fetch our Kaggle data:

In [10]:
if not os.path.exists('data'):
  os.makedirs('data')

# Let's get the Titanic dataset
api.competition_download_files('titanic', path='data')

import zipfile
with zipfile.ZipFile("data/titanic.zip","r") as zip_ref:
    zip_ref.extractall("data")

os.remove('data/titanic.zip')
os.remove('data/gender_submission.csv')

In [11]:
!ls data

test.csv  train.csv


Finally, let's set up some variables for taking the data exploration further:

In [142]:
path = "data"
train_data = "train.csv"



---



## **Exploratory Data Analysis**

EDA can be performed here, where you'll find cells for showing batches, as well as utility functions for displaying certain analytics. It also contains headings to prompt some thinking about possible exploratory approaches. For tabular data in particular, this section will include tree classifiers for model/data fit inspection.

**REMEMBER**: This section is not prescriptive! Add and remove from it as you want and need to.

### **Look at the Data**

In [45]:
# Setup the dataset path
train_path = "{b}/{d}".format(b=path, d=train_data)

# And read it in to a df
df = pd.read_csv(train_path, low_memory=False)

In [46]:
# Set a "reset" df, in case we want to reset the data 
reset_df = df.copy(deep=True)

In [14]:
print(df.columns)
len(df)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


891

We can then set our dependent variable, the one we care about. We can also assign every other feature to an independent variablet set.

In [15]:
dep_var = "Survived"
ind_var = [c for c in df.columns if c != dep_var]

It's always important to take a look at some of the entries themselves so that we can develop a better intuition for what we're working with.

In [47]:
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


We can see `NaN` values, which we'll need to decide how we want to handle, as well as pick out any potentially interesting features based on their values. It may also be useful to pick out "magic features", those which have a strong correlation to the target given some relatively simple transformation.

### **Handling Dates**

Dates often pose a challenge from an encoding point of view. They often have a lot of semantic meaning to us though (did it occur on a weekend? a holiday? etc), so we may want to encode special properties of the dates in the dataset.

In [96]:
# Example from fast.ai Tabular core
date_df = pd.DataFrame({'date': ['2019-12-04', None, '2019-11-15', '2019-10-24']})
date_df = add_datepart(date_df, 'date')
date_df.head()

,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,2019.0,12.0,49.0,4.0,2.0,338.0,False,False,False,False,False,False,1.575418e+09
1,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,NaN
2,2019.0,11.0,46.0,15.0,4.0,319.0,False,False,False,False,False,False,1.573776e+09
3,2019.0,10.0,43.0,24.0,3.0,297.0,False,False,False,False,False,False,1.571875e+09


Further date-related features can be built if there is something specifically relevant to our project. For example, do we care about sales in a specific holiday season, or is a particular virus infection rate seasonal? 

### **Handling Rank in Ordinal Columns**

Some categorical data will be ranked (*ordinal columns*), and for these features it may be useful to tell Pandas about how these categories are ordered.

In [ ]:
# As an example, let's take the passenger class for the Titanic dataset
df['Pclass'].unique()

array([3, 1, 2])

In [ ]:
# export 
def define_ordinal_rank(df, col, ranks):
  """
  Defines ordinal ranking of a column in a dataset

  Args:
    df: DataFrame to define for
    col: Column to define for
    ranks: An array of the ordinal ranking for col
  """
  df[col] = df[col].astype('category')
  df[col].cat.set_categories(ranks, ordered=True, inplace=True)

In [ ]:
define_ordinal_rank(df, 'Pclass', [1, 2, 3])

### **Handling Missing Values**

It's extremely common to find missing content in datasets and it's important to decide how to handle these. Some libraries, like fastai, do have built in handlers for this, but may be different from those found in something like Pandas.

In [117]:
df = reset_df.copy(deep=True)

In [118]:
# We can first find which columns contain NaN values
df.columns[df.isna().any()].tolist()

['Age', 'Cabin', 'Embarked']

We can mark the existence of `NaN` in a row-column pair by creating a new column for each of the columns that contain `NaN`. In each of these new columns, if there was a `NaN` in that row we'll mark it with a 1 and if not we'll mark with a 0.

In [119]:
# export
def mark_nan_cols(df):
  """
  Creates a new column for each NaN column in the df, 
  marking whether the row contained a NaN or not
  """
  nan_cols = df.columns[df.isna().any()].tolist()

  for col in nan_cols:
    df["{}_NaN".format(col)] = np.where(df[col].isnull(), 1, 0)

In [120]:
mark_nan_cols(df)

In [ ]:
# Fill forward
df.fillna(method='ffill', inplace=True)

In [121]:
# Fill with static value
df['Cabin'].fillna("D999", inplace=True)

In [122]:
# Fill with mode
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

In [123]:
# Fill with median
df['Age'].fillna(round(df['Age'].mean()), inplace=True)

In [124]:
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_NaN,Cabin_NaN,Embarked_NaN
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,D999,S,0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C,0,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,D999,S,0,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,0,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,D999,S,0,1,0


In [125]:
# Final check
df.columns[df.isna().any()].tolist()

[]

### **Automated Data Checks and Processing**

Some simple checks can be made on the dataset automatically, depending on what it is you're looking for. In addition, certain types of augmentation or processing can be performed on the data in an automated fashion.

#### **Roulette Target**

A "roulette" occurs when there are duplicate rows with different target values. This makes training much more difficult, as target values may be close to random.

To start, we'll need to define some kind of acceptable amount of duplicates within the system. This should be generally okay as it's possible that some external, non-codified features have an effect on the target variable. So first we define an acceptable proportion of the dataset as duplicates:

In [ ]:
ACCEPTANCE_THRES = .02

And then we have two options. One is to do a simple check on the proportion of duplicates in the dataset and match it against our accepted proportion threshold. We do not consider rows that are full duplicates (that is they duplicate both the row values and the targets) so let's first set up a function to find the relevant duplicates:

In [ ]:
# export
def get_relevant_duplicates(df, dep_var):
  """
  Gets number of duplicates with differing targets
  """
  ind_var = [c for c in df.columns if c != dep_var]

  # We need to sift out full duplicate rows
  poss_dups = df.duplicated(ind_var).sum()
  full_dups = df.duplicated().sum()
  return poss_dups - full_dups

And then we can implement our simple solution:

In [ ]:
# Naive, workable solution
relevant_dups = get_relevant_duplicates(df, dep_var)
is_roulette = (relevant_dups / len(df) * 100) > ACCEPTANCE_THRES

print("Dataset is a roulette:", is_roulette)

Dataset is a roulette: False


This approach can work. A more rigorous approach is to apply a statistical hypothesis test against the accepted threshold and see if that holds up!

For this simple test we claim that the dataset is not a roulette ($H_0$) and perform a [1-proportion test](https://www.tutorialspoint.com/statistics/one_proportion_z_test.htm) to find the associated p-value for this hypothesis. Our alternative hypothesis is that our dataset contains fewer relevant duplicates than our accepted threshold:

In [ ]:
# export
import statsmodels.api as sm

ALPHA = .05

def roulette_test(df, dep_var, threshold):
  """
  Performs a 1-proportion z-test on df to check for roulette. 
  Null hypothesis is that the number of duplicates do not 
  constitute a roulette dataset, in that the number is lower than 
  the acceptance threshold
  """
  relevant_dups = get_relevant_duplicates(df, dep_var)
  if relevant_dups == 0: return False

  _, p_val = sm.stats.proportions_ztest(
      relevant_dups, len(df), len(df)*threshold, 'smaller')

  return p_val > ALPHA

In [ ]:
is_roulette = roulette_test(df, dep_var, ACCEPTANCE_THRES)
print("Dataset is a roulette:", is_roulette)

Dataset is a roulette: False


This check may or may not be relevant depending on the context of the dataset. Consider an actual roulette wheel's results or the results of a series of horse races, which may in fact contain a number of full duplicates that is higher than our threshold. In such a scenario, it would be beneficial to know before pursuing such a data science project further, as the dataset's value entropy may be too high to warrant further work.

#### **Highly Correlated Features**

Some features within the dataset may be highly correlated, and we may want to check for these and then make some decision about them.


### **Basic Data Cleaning**

#### **Removing Outliers**

In [ ]:
# export
def remove_outliers(df, cols=None):
  """
  Removes outlier entries in df 
  """
  if cols:
    w_df = df[cols]
  else:
    w_df = df

  z_scores = np.abs(stats.zscore(w_df, nan_policy='omit'))

  if cols:
    df = pd.concat([df, w_df], axis=1)

  print('z scores', print(len(np.where(z_scores > 3)[0])))
  df = df[(z_scores < 3).all(axis=1)]

  return df

#### **Drop Rare Features**

Certain features may have too few events to provide any meaningful insight to a model, and we can remove these automatically if needed.

In [106]:
# export
def drop_rare_features(df, thres=.9):
  """
  Drops features in the dataset that have 
  events that are too rare to be statistically useful
  """
  rare_f = []
  print("Running with threshold", thres)
  print("")

  for col in df.columns:
    freq = df[col].value_counts(normalize=True)
    sum_freq = df[col].sum()
            
    # should be enough to check whether the most freq is dominant
    if freq.iloc[0] >= thres:
      rare_f.append(col)
      print("\x1b[31m{c}: {v}\x1b[0m".format(c=col, v=freq.iloc[0]))
    else:
      print("{c}: {v}".format(c=col, v=freq.iloc[0]))
  
  df = df.drop(rare_f, axis=1)
  return df

#### **Final Clean Up**

In [ ]:
# Remove outliers
print("DF before:", df.shape)
print("")

df = remove_outliers(df, 'Fare')

print("")
print("DF after removing outliers", df.shape)

In [126]:
# Drop rare features
print("DF before:", len(df.columns))
print("")

df = drop_rare_features(df)

print("")
print("DF after drop rare features:", len(df.columns))

DF before: 15

Running with threshold 0.9

PassengerId: 0.001122334455667789
Survived: 0.6161616161616161
Pclass: 0.5510662177328844
Name: 0.001122334455667789
Sex: 0.6475869809203143
Age: 0.22671156004489337
SibSp: 0.6823793490460157
Parch: 0.7609427609427609
Ticket: 0.007856341189674524
Fare: 0.04826038159371493
Cabin: 0.7710437710437711
Embarked: 0.7250280583613917
Age_NaN: 0.8013468013468014
Cabin_NaN: 0.7710437710437711
Embarked_NaN: 0.9977553310886644

DF after drop rare features 14


In [127]:
print("DF before:", len(df))

ind_vars = [c for c in df.columns if c != dep_var]

# Remove duplicates
df.drop_duplicates(subset=ind_vars, inplace=True)
print("DF after drop duplicates:", len(df))

DF before: 891
DF after drop duplicates: 891


### **Conversion to TabularPandas**

If we want to convert our dataset into a `TabularPandas` dataset from [fast.ai](https://docs.fast.ai/tabular.core.html#TabularPandas) we can do so here, specifying all the preprocessing and splitting we'd require.

In [132]:
# Start by defining procs
procs = [Categorify, Normalize]

#### **Splitting**

If we have timeseries data we'll probably want to manually specify a validation set, as the sequential nature may be important to the understanding of the data:

In [ ]:
# Example splitting for timeseries
cond = (df.saleYear<2011) | (df.saleMonth<10)
train_idx = np.where( cond)[0]
valid_idx = np.where(~cond)[0]

splits = (list(train_idx),list(valid_idx))

Alternatively you can split randomly:

In [131]:
splits = RandomSplitter()(range_of(df))

#### **Final Setup**

For categorical data we'll want to tell `TabularPandas` which columns are categorical and which are continuous:

In [129]:
cont,cat = cont_cat_split(df, dep_var=dep_var)
print("Continuous columns:", cont)
print("Categorical columns:", cat)

Continuous columns: ['PassengerId', 'Age', 'Fare']
Categorical columns: ['Pclass', 'Name', 'Sex', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked', 'Age_NaN', 'Cabin_NaN']


In [133]:
# Finally we set the TabularPandas df up
tdf = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits)

In [138]:
tdf.show(5)

,Pclass,Name,Sex,SibSp,Parch,Ticket,Cabin,Embarked,Age_NaN,Cabin_NaN,PassengerId,Age,Fare,Survived
59,3,"Goodwin, Master. William Frederick",male,5,2,CA 2144,D999,S,0,1,60.0,11.0,46.900002,0
498,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,1,2,113781,C22 C26,S,0,0,499.0,25.0,151.550003,0
287,3,"Naidenoff, Mr. Penko",male,0,0,349206,D999,S,0,1,288.0,22.0,7.895800,0
122,2,"Nasser, Mr. Nicholas",male,1,0,237736,D999,C,0,1,123.0,32.5,30.070801,0
857,1,"Daly, Mr. Peter Denis",male,0,0,113055,E17,S,0,0,858.0,51.0,26.549999,1


In [139]:
tdf.items.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_NaN,Cabin_NaN
59,-1.492656,0,3,300,2,-1.395378,6,3,567,0.291093,116,3,1,2
498,0.218335,0,1,19,1,-0.341892,2,3,38,2.399128,63,3,1,1
287,-0.604032,0,3,571,2,-0.567639,1,1,354,-0.494595,116,3,1,2
122,-1.247115,0,2,576,2,0.222475,2,1,133,-0.047909,116,1,1,2
857,1.617528,1,1,197,2,1.614582,1,1,24,-0.118831,121,3,1,1


In [134]:
len(tdf.train),len(tdf.valid)

(713, 178)

We can also now save the data, as we've probably done a lot of work on it up to this point.

In [143]:
save_pickle("{}/tdf.pkl".format(path),tdf)



---



## **Model**

Model work can be performed here, with utilities to help with cross-validation and architecture construction.



---



## **Inference and Deployment**

Here the model can finally be put to use, as well as exported for deployment in an external application.



---



## **Exports and Clean Up**

Here you can export any cells with the `#export` comment using `notebook2script.py`, as well as cleaning up any environmental changes such as data downloads to a cloud drive.

### **Exports**

In [ ]:
!python notebook2script.py Tabular.ipynb

### **Clean Up**

In [ ]:
# Tear down the data folder
!rm -rf data
!ls